In [20]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset

In [21]:
dt=pd.read_csv(r'filtered_data.csv')

In [3]:
dt.head()

,subject_id,label,chest_ACC_x,chest_ACC_y,chest_ACC_z,chest_ECG,chest_EMG,chest_EDA,chest_Temp,chest_Resp
0,S2,0,0.9554,-0.2220,-0.5580,0.021423,-0.004440,5.250549,30.120758,-1.148987
1,S2,0,0.9258,-0.2216,-0.5538,0.020325,0.004349,5.267334,30.129517,-1.124573
2,S2,0,0.9082,-0.2196,-0.5392,0.016525,0.005173,5.243301,30.138214,-1.152039
3,S2,0,0.8974,-0.2102,-0.5122,0.016708,0.007187,5.249405,30.129517,-1.158142
4,S2,0,0.8882,-0.2036,-0.4824,0.011673,-0.015152,5.286407,30.130950,-1.161194


In [4]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5128356 entries, 0 to 5128355
Data columns (total 10 columns):
 #   Column       Dtype  
---  ------       -----  
 0   subject_id   object 
 1   label        int64  
 2   chest_ACC_x  float64
 3   chest_ACC_y  float64
 4   chest_ACC_z  float64
 5   chest_ECG    float64
 6   chest_EMG    float64
 7   chest_EDA    float64
 8   chest_Temp   float64
 9   chest_Resp   float64
dtypes: float64(8), int64(1), object(1)
memory usage: 391.3+ MB


In [6]:
dt.shape

(5128356, 10)

In [22]:
features = dt[['chest_ACC_x', 'chest_ACC_y', 'chest_ACC_z', 'chest_ECG', 'chest_EMG', 'chest_EDA', 'chest_Temp', 'chest_Resp']]
target = dt['label']

In [23]:
# Convert to tensors
features_tensor = torch.tensor(features.values, dtype=torch.float32)
target_tensor = torch.tensor(target.values, dtype=torch.long)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(features_tensor, target_tensor, test_size=0.2, random_state=42)

In [25]:
# Create PyTorch datasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [26]:
# Create DataLoader for batch processing
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [14]:
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # LSTM expects input of shape (batch_size, sequence_length, input_size)
        x = x.unsqueeze(1)  # Add a sequence length of 1
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Use the last output for classification
        return out


In [15]:
class AttentionModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(AttentionModel, self).__init__()
        self.attention = nn.Linear(hidden_size, 1)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Apply first linear layer
        x = torch.relu(self.fc1(x))
        # Compute attention weights
        attention_weights = torch.softmax(self.attention(x), dim=1)
        # Apply attention weights
        x = (attention_weights * x).sum(dim=1)
        return self.fc2(x)


In [31]:
from torch.nn import Transformer

class TransformerModel(nn.Module):
    def __init__(self, input_size, num_heads, hidden_size, output_size, num_layers=2):
        super(TransformerModel, self).__init__()
        self.transformer = Transformer(d_model=input_size, nhead=num_heads, num_encoder_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(input_size, output_size)  # Final fully connected layer for classification
    
    def forward(self, x):
        # Transformer requires a 3D input tensor of shape (batch_size, sequence_length, input_size)
        x = self.transformer(x, x)  # Using same tensor as input and target to simplify
        x = x.mean(dim=1)  # Pooling, take the mean across the sequence dimension
        return self.fc(x)  # Apply final fully connected layer

In [29]:
import torch.nn as nn
def train_model(model, train_loader, test_loader, num_epochs=5, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Evaluate the model
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                outputs = model(X_batch)
                _, predicted = torch.max(outputs, 1)
                total += y_batch.size(0)
                correct += (predicted == y_batch).sum().item()
        
        accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/10000:.4f}, Accuracy: {accuracy:.2f}%')


In [18]:
input_size = features.shape[1]
hidden_size = 128
num_layers = 2
output_size = len(target.unique())  # Number of unique labels

lstm_model = LSTMModel(input_size, hidden_size, num_layers, output_size)
train_model(lstm_model, train_loader, test_loader)


Epoch [1/5], Loss: 10472.0328, Accuracy: 96.34%
Epoch [2/5], Loss: 5775.6879, Accuracy: 96.87%
Epoch [3/5], Loss: 4714.0968, Accuracy: 97.56%
Epoch [4/5], Loss: 4154.9206, Accuracy: 97.21%
Epoch [5/5], Loss: 3788.4870, Accuracy: 97.92%


In [13]:
features.shape[1]

8

In [15]:
len(target.unique())

5

In [17]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset



class AttentionModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(AttentionModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)  # LSTM layer
        self.fc1 = nn.Linear(hidden_size, hidden_size)  # Second hidden layer
        self.fc2 = nn.Linear(hidden_size, output_size)  # Output layer
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer

    def forward(self, x):
        x = x.unsqueeze(1)  # Add sequence dimension
        x, _ = self.lstm(x)  # Pass through LSTM
        x = x[:, -1, :]  # Get last output from LSTM
        x = torch.relu(self.fc1(x))  # First layer with ReLU activation
        x = self.dropout(x)  # Apply dropout
        return self.fc2(x)  # Output layer


    def calculate_attention_weights(self, x):
        # Scaled dot-product attention example
        attention_scores = torch.matmul(x, x.transpose(1, 2)) / (x.size(-1) ** 0.5)
        attention_weights = torch.softmax(attention_scores, dim=-1)
        return attention_weights

# Instantiate the model
input_size = features.shape[1]  # Number of input features (8)
hidden_size = 64  # Hidden size (you can adjust this)
output_size = len(target.unique())  # Number of unique labels (5)

Attention_Model = AttentionModel(input_size, hidden_size, output_size,)


# Run the training
train_model(Attention_Model, train_loader, test_loader)



Epoch [1/5], Loss: 1.8294, Accuracy: 94.44%
Epoch [2/5], Loss: 1.3374, Accuracy: 93.75%
Epoch [3/5], Loss: 1.2005, Accuracy: 95.62%
Epoch [4/5], Loss: 1.1075, Accuracy: 95.77%
Epoch [5/5], Loss: 1.0469, Accuracy: 95.70%


In [19]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

class EnhancedAttentionModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(EnhancedAttentionModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True, dropout=0.3)  # Double LSTM layers
        self.attention_fc = nn.Linear(hidden_size, hidden_size)  # For attention mechanism
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add sequence dimension
        lstm_out, _ = self.lstm(x)
        attention_weights = torch.softmax(self.attention_fc(lstm_out[:, -1, :]), dim=1)  # Attention on LSTM output
        x = attention_weights * lstm_out[:, -1, :]  # Weighted sum
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)



    def calculate_attention_weights(self, x):
        # Scaled dot-product attention example
        attention_scores = torch.matmul(x, x.transpose(1, 2)) / (x.size(-1) ** 0.5)
        attention_weights = torch.softmax(attention_scores, dim=-1)
        return attention_weights

# Instantiate the model
input_size = features.shape[1]  # Number of input features (8)
hidden_size = 64  # Hidden size (you can adjust this)
output_size = len(target.unique())  # Number of unique labels (5)

Attention_Model = EnhancedAttentionModel(input_size, hidden_size, output_size,)


# Run the training
train_model(Attention_Model, train_loader, test_loader)



Epoch [1/5], Loss: 2.6155, Accuracy: 93.26%
Epoch [2/5], Loss: 1.2928, Accuracy: 94.99%
Epoch [3/5], Loss: 1.1116, Accuracy: 95.24%
Epoch [4/5], Loss: 1.0207, Accuracy: 95.22%
Epoch [5/5], Loss: 0.9559, Accuracy: 95.19%


In [27]:
import torch
import torch.nn as nn
import math
from torch.nn import Transformer

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class AdvancedTransformerModel(nn.Module):
    def __init__(self, input_size, num_heads, hidden_size, output_size, num_layers=2, dropout=0.1):
        super(AdvancedTransformerModel, self).__init__()
        self.embedding = nn.Linear(input_size, hidden_size)  # Embedding layer to project input
        self.pos_encoder = PositionalEncoding(hidden_size)  # Positional encoding
        self.transformer = Transformer(
            d_model=hidden_size,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=hidden_size * 4,
            dropout=dropout,
            batch_first=True  # Ensures compatibility with batch dimension
        )
        self.fc = nn.Linear(hidden_size, output_size)  # Classification head
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(hidden_size)  # Normalization layer

    def forward(self, x):
        # Embedding
        x = self.embedding(x)
        x = self.pos_encoder(x)  # Add positional encoding
        
        # Transformer Encoder
        x = self.transformer(x, x)  # Self-attention applied here
        
        # Pooling: Take the mean of the sequence for classification
        x = x.mean(dim=1)  # Mean pooling across sequence length

        # Final layers
        x = self.layer_norm(x)  # Layer normalization
        x = self.dropout(x)     # Dropout for regularization
        return self.fc(x)       # Classification output

# Instantiate the model with parameters
input_size = 8  # Number of input features
num_heads = 4  # Number of attention heads
hidden_size = 128  # Hidden size for embedding and transformer layers
output_size = 5  # Number of output classes
num_layers = 3  # Number of transformer layers
dropout = 0.3  # Dropout rate

model = AdvancedTransformerModel(input_size, num_heads, hidden_size, output_size, num_layers, dropout)

# Print the model to inspect architecture
print(model)


AdvancedTransformerModel(
  (embedding): Linear(in_features=8, out_features=128, bias=True)
  (pos_encoder): PositionalEncoding()
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
          )
          (linear1): Linear(in_features=128, out_features=512, bias=True)
          (dropout): Dropout(p=0.3, inplace=False)
          (linear2): Linear(in_features=512, out_features=128, bias=True)
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.3, inplace=False)
          (dropout2): Dropout(p=0.3, inplace=False)
        )
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(


In [28]:
# Run the training
train_model(model, train_loader, test_loader, num_epochs=5, learning_rate=0.001)

KeyboardInterrupt: 

In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import Transformer

class TransformerModel(nn.Module):
    def __init__(self, input_size, num_heads, hidden_size, output_size, num_layers=2):
        super(TransformerModel, self).__init__()
        
        self.input_fc = nn.Linear(input_size, hidden_size)  # Project input to hidden size
        self.transformer = Transformer(d_model=hidden_size, nhead=num_heads, num_encoder_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)  # Final output layer

    def forward(self, x):
        # x: [batch_size, seq_len, input_size]
        x = x.unsqueeze(1)  # Add sequence length dimension: [batch_size, seq_len=1, input_size]
        x = self.input_fc(x)  # [batch_size, seq_len=1, hidden_size]
        x = self.transformer(x, x)  # Pass through Transformer
        
        # Use the last output of the sequence
        x = x[:, -1, :]  # Take the last output for classification: [batch_size, hidden_size]
        
        return self.fc(x)  # [batch_size, output_size]

# Training function remains the same
def train_model(model, train_loader, test_loader, num_epochs=5, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.float(), labels.long()  # Ensure correct types
            
            optimizer.zero_grad()
            outputs = model(inputs)  # Expected shape: [batch_size, output_size]

            # Debugging shapes
            #print(f"Inputs shape: {inputs.shape}, Outputs shape: {outputs.shape}, Labels shape: {labels.shape}")

            # Ensure the shapes match for the loss calculation
            assert outputs.shape[0] == labels.shape[0], "Output and labels batch sizes do not match!"
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        train_accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%")
        
        # Evaluate on test set after each epoch
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.float(), labels.long()
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        test_accuracy = 100 * correct / total
        print(f"Test Accuracy after epoch {epoch+1}: {test_accuracy:.2f}%\n")

# Model and training parameters
input_size = 8       # Number of input features
num_heads = 4        # Number of attention heads
hidden_size = 64     # Hidden size for embedding and transformer layers
output_size = 5      # Number of output classes
num_layers = 2       # Number of transformer layers

# Initialize and train the model
model = TransformerModel(input_size, num_heads, hidden_size, output_size, num_layers)
train_model(model, train_loader, test_loader, num_epochs=5, learning_rate=0.001)


Epoch [1/5], Loss: 0.3052, Accuracy: 87.74%
Test Accuracy after epoch 1: 93.06%

Epoch [2/5], Loss: 0.1757, Accuracy: 93.51%
Test Accuracy after epoch 2: 95.15%

Epoch [3/5], Loss: 0.1568, Accuracy: 94.23%
Test Accuracy after epoch 3: 95.76%

Epoch [4/5], Loss: 0.1351, Accuracy: 95.05%
Test Accuracy after epoch 4: 95.40%

Epoch [5/5], Loss: 0.1205, Accuracy: 95.55%
Test Accuracy after epoch 5: 96.19%

